# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_df = pd.read_csv("output.csv")
weather_df.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country
0,0,caravelas,-17.7125,-39.2481,80.51,59,0,9.53,1627476317,BR
1,1,lakhimpur,27.9500,80.7667,86.20,74,100,6.26,1627476319,IN
2,2,saeby,57.3319,10.5225,69.96,79,72,9.42,1627476321,DK
3,3,katangli,51.7102,143.2326,62.01,90,95,4.00,1627476322,RU
4,4,mataura,-46.1927,168.8643,46.47,98,40,3.56,1627476322,NZ


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)
# Create initial figure
fig = gmaps.figure()

# Create Heatmap layer
locations = weather_df[["Lat", "Lng"]]
heatmap_layer = gmaps.heatmap_layer(locations, weights=weather_df["Humidity"])
fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [23]:
#create good weather conditions for holiday
idx = np.where((weather_df['Wind Speed']< 10)&
               (weather_df['Max Temp']>70)&
               (weather_df['Max Temp']<80)&
               (weather_df['Cloudiness']==0))


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [26]:
#store in hotel_df dataframe
hotel_df = weather_df.loc[idx]
#create new column
hotel_df['Hotel Name'] = ""
hotel_df

,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
19,19,wakkanai,45.4094,141.6739,73.27,89,0,7.31,1627476328,JP,
225,225,vila velha,-20.3297,-40.2925,77.97,67,0,9.22,1627476382,BR,
280,280,yerbogachen,61.2767,108.0108,75.16,58,0,6.33,1627476152,RU,
291,291,pingliang,35.5392,106.6861,71.73,49,0,3.89,1627476399,CN,
312,312,tekeli,44.8300,78.8239,74.39,31,0,8.16,1627476404,KZ,
385,385,masvingo,-20.0637,30.8277,71.08,30,0,2.71,1627476417,ZW,
423,423,gotsu,35.0000,132.2167,74.41,88,0,4.61,1627476434,JP,
431,431,huron,41.1501,-82.5499,76.10,88,0,5.82,1627476437,US,
547,547,vila,42.0304,-8.1588,74.75,42,0,6.73,1627476453,PT,


In [29]:
#copy lat and longitude to a list zipped together
lat =hotel_df['Lat'].to_list()
lng = hotel_df['Lng'].to_list()
lat_lng = list(zip(lat, lng))

#create variables for params
target_coordinates = lat_lng
target_radius = 5000
target_type = "hotel"

#set up a parameters dictionary
params = {
    "location": target_coordinates,
    "radius": target_radius,
    "type": target_type,
    "key": g_key
}

#requesting the hotels and saving the first hotel in hotel name
for index, row in hotel_df.iterrows():
    lat = row['Lat']
    lng = row['Lng']
    
    params['location'] = f"{lat},{lng}"
    # base url
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    response = requests.get(base_url, params=params)
    hotel_address = response.json()
    try:
        hotel_df.loc[index, "Hotel Name"] = hotel_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/Result ... skipping")
    
hotel_df


,Unnamed: 0,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Country,Hotel Name
19,19,wakkanai,45.4094,141.6739,73.27,89,0,7.31,1627476328,JP,Wakkanai
225,225,vila velha,-20.3297,-40.2925,77.97,67,0,9.22,1627476382,BR,Vila Velha
280,280,yerbogachen,61.2767,108.0108,75.16,58,0,6.33,1627476152,RU,Yerbogachen
291,291,pingliang,35.5392,106.6861,71.73,49,0,3.89,1627476399,CN,Pingliang
312,312,tekeli,44.8300,78.8239,74.39,31,0,8.16,1627476404,KZ,Tekeli
385,385,masvingo,-20.0637,30.8277,71.08,30,0,2.71,1627476417,ZW,Masvingo
423,423,gotsu,35.0000,132.2167,74.41,88,0,4.61,1627476434,JP,Gotsu
431,431,huron,41.1501,-82.5499,76.10,88,0,5.82,1627476437,US,Olena
547,547,vila,42.0304,-8.1588,74.75,42,0,6.73,1627476453,PT,Castro Laboreiro


In [47]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [48]:
# Add marker layer ontop of heat map
fig = gmaps.figure()

marker_locations = hotel_df[['Lat', 'Lng']]
hotel_name = hotel_df["Hotel Name"].tolist()
max_temp = hotel_df["Max Temp"]

#create two layers
heatmap_layer = gmaps.heatmap_layer(marker_locations, weights=hotel_df["Humidity"])
markers = gmaps.marker_layer(marker_locations, info_box_content = hotel_info)

# Display figure
fig.add_layer(markers)
fig.add_layer(heatmap_layer)
fig

Figure(layout=FigureLayout(height='420px'))